In [ ]:
# DeepSense Scenario 31: 64 Beams - Transfomers - MaxViT!

# Average Top-1 accuracy 0.46438746438746437
# Average Top-3 accuracy 0.8262108262108262
# Average Top-5 accuracy 0.9202279202279202
# Average Top-7 accuracy 0.9458689458689459
# Average Top-9 accuracy 0.9558404558404558
# Average Top-11 accuracy 0.9629629629629629
# Average Top-13 accuracy 0.9715099715099715
# Average Top-15 accuracy 0.9715099715099715

In [ ]:
# !pip install timm

In [1]:
import os
import datetime
import shutil
import torch
import torch as t
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transf
from torchsummary import summary
from torch.utils.model_zoo import load_url as load_state_dict_from_url

import numpy as np
import pandas as pd

import random
from skimage import io

import torch
import torch.nn as nn
import torch.optim as optimizer
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transf
import pandas as pd
from skimage import io
from timm import create_model

In [2]:
# Save directory
# year month day
dayTime = datetime.datetime.now().strftime('%m-%d-%Y')
# Minutes and seconds
hourTime = datetime.datetime.now().strftime('%H_%M')
print(dayTime + '\n' + hourTime)

pwd = os.getcwd() + '//' + 'saved_folder' + '//' + dayTime + '_' + hourTime
print(pwd)
isExists = os.path.exists(pwd)
if not isExists:
    os.makedirs(pwd)

save_directory = pwd + '//' + 'saved_analysis_files'
checkpoint_directory = pwd + '//' + 'checkpoint'

isExists = os.path.exists(save_directory)
if not isExists:
    os.makedirs(save_directory)

    isExists = os.path.exists(checkpoint_directory)
if not isExists:
    os.makedirs(checkpoint_directory)

07-25-2024
17_40
C:\Users\Baqer\Desktop\V2X_CNN_All\Scenario31_64-Beams\Main_Folder//saved_folder//07-25-2024_17_40


In [3]:
# Data Feeding: Create data sample list
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        img_paths = row.values[1:3]
        data_samples.append(img_paths)
    return data_samples

class DataFeed(Dataset):
    '''
    A class retrieving a tuple of (image,label). It can handle the case
    of empty classes (empty folders).
    '''
    def __init__(self,root_dir, nat_sort = False, transform=None, init_shuflle = True):
        self.root = root_dir
        self.samples = create_samples(self.root,shuffle=init_shuflle,nat_sort=nat_sort)
        self.transform = transform


    def __len__(self):
        return len( self.samples )

    def __getitem__(self, idx):
        sample = self.samples[idx]
        img = io.imread(sample[0])
        if self.transform:
            img = self.transform(img)
        label = sample[1]
        return (img,label)

In [4]:
def create_samples(root, shuffle=False, nat_sort=False):
    f = pd.read_csv(root)
    data_samples = []
    pred_val = []
    for idx, row in f.iterrows():
        img_paths = row.values[1:3]
        data_samples.append(img_paths)
    return data_samples

class DataFeed(Dataset):
    def __init__(self, root_dir, nat_sort=False, transform=None, init_shuflle=True):
        self.root = root_dir
        self.samples = create_samples(self.root, shuffle=init_shuflle, nat_sort=nat_sort)
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx]
        img = io.imread(sample[0])
        if self.transform:
            img = self.transform(img)
        label = sample[1]
        return (img, label)

In [ ]:
# Training, Testing, and Validation!

In [5]:
batch_size = 16 # 8
val_batch_size = 1
lr = 1e-3
decay = 1e-4
num_epochs = 20
train_size = [1]

In [6]:
img_resize = transf.Resize((224, 224))
img_norm = transf.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
proc_pipe = transf.Compose(
    [transf.ToPILImage(),
     img_resize,
     transf.ToTensor(),
     img_norm]
)

In [ ]:
# Training and Validation

In [7]:
train_dir = 'scenario31_64_img_beam_train.csv'
val_dir = 'scenario31_64_img_beam_val.csv'

train_loader = DataLoader(DataFeed(train_dir, transform=proc_pipe),
                          batch_size=batch_size,
                          shuffle=False)
val_loader = DataLoader(DataFeed(val_dir, transform=proc_pipe),
                        batch_size=val_batch_size,
                        shuffle=False)

In [8]:
# !pip install GPUtil

import torch
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()

free_gpu_cache()

Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  8% | 30% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 |  8% | 32% |


In [9]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [10]:
import torch

print("CUDA Available:", torch.cuda.is_available())
print("CUDA Device Count:", torch.cuda.device_count())
print("CUDA Device Name:", torch.cuda.get_device_name(0))

CUDA Available: True
CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce RTX 2080 Super with Max-Q Design


In [11]:
torch.cuda.memory_allocated()

0

In [12]:
torch.cuda.max_memory_allocated()

0

In [13]:
import torch.cuda as cuda

with cuda.device(0):
    acc_loss = 0
    itr = []
    for idx, n in enumerate(train_size):
        print('```````````````````````````````````````````````````````')
        print('Training size is {}'.format(n))
        # Build the network:
        # net = resnet50(pretrained=True, progress=True, num_classes=64)

        net = create_model('maxvit_tiny_tf_224', pretrained=True, num_classes=65)
        net = net.cuda()
        summary(net.cuda(), (3, 224, 224))

        # Optimization parameters:
        criterion = nn.CrossEntropyLoss()
        opt = torch.optim.Adam(net.parameters(), lr=lr, weight_decay=decay)
        LR_sch = torch.optim.lr_scheduler.MultiStepLR(opt, [4, 8, 12], gamma=0.1, last_epoch=-1)

        count = 0
        running_loss = []
        running_top1_acc = []
        running_top3_acc = []
        running_top5_acc = []
        running_top7_acc = []
        running_top9_acc = []
        running_top11_acc = []
        running_top13_acc = []
        running_top15_acc = []

        best_accuracy = 0

        for epoch in range(num_epochs):
            print('Epoch No. ' + str(epoch + 1))
            skipped_batches = 0
            epoch_train_loss = 0  # To track the training loss for the epoch
            for tr_count, (img, label) in enumerate(train_loader):
                net.train()
                x = img.cuda()
                opt.zero_grad()
                label = label.cuda()
                out = net(x)
                L = criterion(out, label)
                L.backward()
                opt.step()
                batch_loss = L.item()
                acc_loss += batch_loss
                epoch_train_loss += batch_loss  # Accumulate batch loss for the epoch
                count += 1
                if count % 10 == 0:
                    print('Training-Batch No.' + str(count))
                    running_loss.append(batch_loss)
                    itr.append(count)
                    print('Loss = ' + str(running_loss[-1]))

            epoch_train_loss /= len(train_loader)  # Calculate average training loss for the epoch
            print(f'Epoch {epoch + 1} Training Loss: {epoch_train_loss:.4f}')

            print('Start validation')
            ave_top1_acc = 0
            ave_top3_acc = 0
            ave_top5_acc = 0
            ave_top7_acc = 0
            ave_top9_acc = 0
            ave_top11_acc = 0
            ave_top13_acc = 0
            ave_top15_acc = 0
            val_loss = 0  # To track the validation loss
            ind_ten = torch.tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
            top1_pred_out = []
            top3_pred_out = []
            top5_pred_out = []
            top7_pred_out = []
            top9_pred_out = []
            top11_pred_out = []
            top13_pred_out = []
            top15_pred_out = []
            gt_beam = []
            total_count = 0
            for val_count, (imgs, labels) in enumerate(val_loader):
                net.eval()
                x = imgs.cuda()
                labels = labels.cuda()
                total_count += labels.size(0)
                out = net(x)
                _, top_1_pred = torch.max(out, dim=1)

                val_batch_loss = criterion(out, labels).item()  # Calculate validation loss for the batch
                val_loss += val_batch_loss  # Accumulate batch loss for the epoch

                gt_beam.append(labels.detach().cpu().numpy()[0])

                top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
                sorted_out = torch.argsort(out, dim=1, descending=True)

                top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
                top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0])

                top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
                top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0])

                top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
                top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0])

                top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
                top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0])

                top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
                top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0])

                top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
                top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0])

                top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
                top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0])

                reshaped_labels = labels.reshape((labels.shape[0], 1))
                tiled_3_labels = reshaped_labels.repeat(1, 3)
                tiled_5_labels = reshaped_labels.repeat(1, 5)
                tiled_7_labels = reshaped_labels.repeat(1, 7)
                tiled_9_labels = reshaped_labels.repeat(1, 9)
                tiled_11_labels = reshaped_labels.repeat(1, 11)
                tiled_13_labels = reshaped_labels.repeat(1, 13)
                tiled_15_labels = reshaped_labels.repeat(1, 15)

                batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
                batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
                batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
                batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
                batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
                batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
                batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
                batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

                ave_top1_acc += batch_top1_acc.item()
                ave_top3_acc += batch_top3_acc.item()
                ave_top5_acc += batch_top5_acc.item()
                ave_top7_acc += batch_top7_acc.item()
                ave_top9_acc += batch_top9_acc.item()
                ave_top11_acc += batch_top11_acc.item()
                ave_top13_acc += batch_top13_acc.item()
                ave_top15_acc += batch_top15_acc.item()

            val_loss /= len(val_loader)  # Calculate average validation loss for the epoch
            print(f'Epoch {epoch + 1} Validation Loss: {val_loss:.4f}')

            print("total test examples are", total_count)
            running_top1_acc.append(ave_top1_acc / total_count)
            running_top3_acc.append(ave_top3_acc / total_count)
            running_top5_acc.append(ave_top5_acc / total_count)
            running_top7_acc.append(ave_top7_acc / total_count)
            running_top9_acc.append(ave_top9_acc / total_count)
            running_top11_acc.append(ave_top11_acc / total_count)
            running_top13_acc.append(ave_top13_acc / total_count)
            running_top15_acc.append(ave_top15_acc / total_count)

            print('Training_size {}--No. of skipped batches {}'.format(n, skipped_batches))
            print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
            print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
            print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
            print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
            print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
            print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
            print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
            print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

            cur_accuracy = running_top1_acc[-1]

            print("current acc", cur_accuracy)
            print("best acc", best_accuracy)
            if cur_accuracy > best_accuracy:
                print("Saving the best model")
                net_name = checkpoint_directory + '//' + '/maxvit_64_beam'
                torch.save(net.state_dict(), net_name)
                best_accuracy = cur_accuracy
            print("updated best accuracy", best_accuracy)


        print("Saving the predicted value in a csv file")
        file_to_save = f'{save_directory}//topk_pred_beam_val_after_{epoch + 1}th_epoch.csv'
        indx = np.arange(1, len(top1_pred_out) + 1, 1)
        df1 = pd.DataFrame()
        df1['index'] = indx
        df1['link_status'] = gt_beam
        df1['top1_pred'] = top1_pred_out
        df1['top3_pred'] = top3_pred_out
        df1['top5_pred'] = top5_pred_out
        df1['top7_pred'] = top7_pred_out
        df1['top9_pred'] = top9_pred_out
        df1['top11_pred'] = top11_pred_out
        df1['top13_pred'] = top13_pred_out
        df1['top15_pred'] = top15_pred_out
        df1.to_csv(file_to_save, index=False)

        LR_sch.step()

```````````````````````````````````````````````````````
Training size is 1
Layer (type:depth-idx)                   Output Shape              Param #
├─Stem: 1-1                              [-1, 64, 112, 112]        --
|    └─Conv2dSame: 2-1                   [-1, 64, 112, 112]        1,792
|    └─BatchNormAct2d: 2-2               [-1, 64, 112, 112]        --
|    |    └─Identity: 3-1                [-1, 64, 112, 112]        --
|    |    └─GELUTanh: 3-2                [-1, 64, 112, 112]        --
|    └─Conv2d: 2-3                       [-1, 64, 112, 112]        36,928
├─Sequential: 1-2                        [-1, 512, 7, 7]           --
|    └─MaxxVitStage: 2-4                 [-1, 64, 56, 56]          --
|    |    └─Sequential: 3-3              [-1, 64, 56, 56]          290,792
|    └─MaxxVitStage: 2-5                 [-1, 128, 28, 28]         --
|    |    └─Sequential: 3-4              [-1, 128, 28, 28]         1,114,064
|    └─MaxxVitStage: 2-6                 [-1, 256, 14, 14]   

In [14]:
print('Start Validation')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = torch.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
gt_beam = []
total_count = 0

for val_count, (imgs, labels) in enumerate(val_loader):
    net.eval()
    x = imgs.cuda()
    labels = labels.cuda()
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = torch.max(out, dim=1)

    gt_beam.append(labels.detach().cpu().numpy()[0])

    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
    sorted_out = torch.argsort(out, dim=1, descending=True)

    top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
    batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
    batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
    batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
    batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
    batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
    batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
    batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

Start Validation
total examples are 2103
Average Top-1 accuracy 0.44935805991440797
Average Top-3 accuracy 0.8302425106990015
Average Top-5 accuracy 0.920114122681883
Average Top-7 accuracy 0.9396100808368997
Average Top-9 accuracy 0.9510223490252021
Average Top-11 accuracy 0.9591060389919163
Average Top-13 accuracy 0.9638611507370424
Average Top-15 accuracy 0.9681407513076558


In [15]:
print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval.csv'

indx = np.arange(1, len(top1_pred_out)+1, 1)
df2 = pd.DataFrame()
df2['index'] = indx
df2['link_status'] = gt_beam  # Add the link_status column

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Saving the predicted value in a csv file


In [ ]:
# Testing

In [16]:
# Load the model checkpoint
test_dir = './scenario31_64_img_beam_test.csv'

# Load the test data
test_data = pd.read_csv(test_dir)

# Extract the 'unit1_pwr1_best-beam' data and convert it to a list
link_status_data = test_data['original_unit1_pwr_best-beam'].tolist()
org = test_data['original_index'].tolist()
pwr_60ghz = test_data['original_unit1_pwr'].tolist()

# Load the model checkpoint and prepare for evaluation
checkpoint_path = f'{checkpoint_directory}/maxvit_64_beam'  # Update the checkpoint path
net = create_model('maxvit_tiny_tf_224', pretrained=False, num_classes=65)  # Ensure the model is instantiated correctly
net.load_state_dict(torch.load(checkpoint_path))
net.eval()
net = net.cuda()

In [17]:
test_loader = DataLoader(DataFeed(test_dir, transform=proc_pipe),
                        batch_size=val_batch_size,
                        #num_workers=8,
                        shuffle=False)

In [18]:
print('Start Testing')
ave_top1_acc = 0
ave_top3_acc = 0
ave_top5_acc = 0
ave_top7_acc = 0
ave_top9_acc = 0
ave_top11_acc = 0
ave_top13_acc = 0
ave_top15_acc = 0
ind_ten = torch.as_tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], device='cuda:0')
top1_pred_out = []
top3_pred_out = []
top5_pred_out = []
top7_pred_out = []
top9_pred_out = []
top11_pred_out = []
top13_pred_out = []
top15_pred_out = []
gt_beam = []
total_count = 0

for val_count, (imgs, labels) in enumerate(test_loader):
    net.eval()
    x = imgs.cuda()
    labels = labels.cuda()
    total_count += labels.size(0)
    out = net.forward(x)
    _, top_1_pred = torch.max(out, dim=1)

    gt_beam.append(labels.detach().cpu().numpy()[0])

    top1_pred_out.append(top_1_pred.detach().cpu().numpy()[0])
    sorted_out = torch.argsort(out, dim=1, descending=True)

    top_3_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:3])
    top3_pred_out.append(top_3_pred.detach().cpu().numpy()[0].tolist())

    top_5_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:5])
    top5_pred_out.append(top_5_pred.detach().cpu().numpy()[0].tolist())

    top_7_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:7])
    top7_pred_out.append(top_7_pred.detach().cpu().numpy()[0].tolist())

    top_9_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:9])
    top9_pred_out.append(top_9_pred.detach().cpu().numpy()[0].tolist())

    top_11_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:11])
    top11_pred_out.append(top_11_pred.detach().cpu().numpy()[0].tolist())

    top_13_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:13])
    top13_pred_out.append(top_13_pred.detach().cpu().numpy()[0].tolist())

    top_15_pred = torch.index_select(sorted_out, dim=1, index=ind_ten[0:15])
    top15_pred_out.append(top_15_pred.detach().cpu().numpy()[0].tolist())

    reshaped_labels = labels.reshape((labels.shape[0], 1))
    tiled_3_labels = reshaped_labels.repeat(1, 3)
    tiled_5_labels = reshaped_labels.repeat(1, 5)
    tiled_7_labels = reshaped_labels.repeat(1, 7)
    tiled_9_labels = reshaped_labels.repeat(1, 9)
    tiled_11_labels = reshaped_labels.repeat(1, 11)
    tiled_13_labels = reshaped_labels.repeat(1, 13)
    tiled_15_labels = reshaped_labels.repeat(1, 15)

    batch_top1_acc = torch.sum(top_1_pred == labels, dtype=torch.float32)
    batch_top3_acc = torch.sum(top_3_pred == tiled_3_labels, dtype=torch.float32)
    batch_top5_acc = torch.sum(top_5_pred == tiled_5_labels, dtype=torch.float32)
    batch_top7_acc = torch.sum(top_7_pred == tiled_7_labels, dtype=torch.float32)
    batch_top9_acc = torch.sum(top_9_pred == tiled_9_labels, dtype=torch.float32)
    batch_top11_acc = torch.sum(top_11_pred == tiled_11_labels, dtype=torch.float32)
    batch_top13_acc = torch.sum(top_13_pred == tiled_13_labels, dtype=torch.float32)
    batch_top15_acc = torch.sum(top_15_pred == tiled_15_labels, dtype=torch.float32)

    ave_top1_acc += batch_top1_acc.item()
    ave_top3_acc += batch_top3_acc.item()
    ave_top5_acc += batch_top5_acc.item()
    ave_top7_acc += batch_top7_acc.item()
    ave_top9_acc += batch_top9_acc.item()
    ave_top11_acc += batch_top11_acc.item()
    ave_top13_acc += batch_top13_acc.item()
    ave_top15_acc += batch_top15_acc.item()

print("total test examples are", total_count)
running_top1_acc.append(ave_top1_acc / total_count)
running_top3_acc.append(ave_top3_acc / total_count)
running_top5_acc.append(ave_top5_acc / total_count)
running_top7_acc.append(ave_top7_acc / total_count)
running_top9_acc.append(ave_top9_acc / total_count)
running_top11_acc.append(ave_top11_acc / total_count)
running_top13_acc.append(ave_top13_acc / total_count)
running_top15_acc.append(ave_top15_acc / total_count)

print('Average Top-1 accuracy {}'.format(running_top1_acc[-1]))
print('Average Top-3 accuracy {}'.format(running_top3_acc[-1]))
print('Average Top-5 accuracy {}'.format(running_top5_acc[-1]))
print('Average Top-7 accuracy {}'.format(running_top7_acc[-1]))
print('Average Top-9 accuracy {}'.format(running_top9_acc[-1]))
print('Average Top-11 accuracy {}'.format(running_top11_acc[-1]))
print('Average Top-13 accuracy {}'.format(running_top13_acc[-1]))
print('Average Top-15 accuracy {}'.format(running_top15_acc[-1]))

print("Saving the predicted value in a csv file")
file_to_save = f'{save_directory}//best_epoch_eval_Test.csv'

indx = test_data.index + 1
df2 = pd.DataFrame()
df2['index'] = org
df2['link_status'] = link_status_data
df2['original_unit1_pwr'] = pwr_60ghz

df2['top1_pred'] = top1_pred_out
df2['top3_pred'] = top3_pred_out
df2['top5_pred'] = top5_pred_out
df2['top7_pred'] = top7_pred_out
df2['top9_pred'] = top9_pred_out
df2['top11_pred'] = top11_pred_out
df2['top13_pred'] = top13_pred_out
df2['top15_pred'] = top15_pred_out
df2.to_csv(file_to_save, index=False)

Start Testing
total test examples are 702
Average Top-1 accuracy 0.46438746438746437
Average Top-3 accuracy 0.8262108262108262
Average Top-5 accuracy 0.9202279202279202
Average Top-7 accuracy 0.9458689458689459
Average Top-9 accuracy 0.9558404558404558
Average Top-11 accuracy 0.9629629629629629
Average Top-13 accuracy 0.9715099715099715
Average Top-15 accuracy 0.9715099715099715
Saving the predicted value in a csv file
